In [1]:
import re, os, glob, nltk
import pandas as pd
import feather
from collections import Counter
from nltk import bigrams, ngrams
from nltk.stem import WordNetLemmatizer
from nltk.collocations import *
from nltk.tokenize import word_tokenize

In [2]:

stop_words = ['а', 'в', 'г', 'е', 'ж', 'и', 'к', 'м', 'о', 'с', 'т', 'у', 'я', 'бы', 'во', 'вы', 'да', 'до', 'ее', 'ей', 'ею', 'её', 'же', 'за', 'из', 'им', 'их', 'ли', 'мы', 'на', 'не', 'ни', 'но', 'ну', 'нх', 'об', 'он', 'от', 'по', 'со', 'та', 'те', 'то', 'ту', 'ты', 'уж', 'без', 'был', 'вам', 'вас', 'ваш', 'вон', 'вот', 'все', 'всю', 'вся', 'всё', 'где', 'год', 'два', 'две', 'дел', 'для', 'его', 'ему', 'еще', 'ещё', 'или', 'ими', 'имя', 'как', 'кем', 'ком', 'кто', 'лет', 'мне', 'мог', 'мож', 'мои', 'мой', 'мор', 'моя', 'моё', 'над', 'нам', 'нас', 'наш', 'нее', 'ней', 'нем', 'нет', 'нею', 'неё', 'них', 'оба', 'она', 'они', 'оно', 'под', 'пор', 'при', 'про', 'раз', 'сам', 'сих', 'так', 'там', 'тем', 'тех', 'том', 'тот', 'тою', 'три', 'тут', 'уже', 'чем', 'что', 'эта', 'эти', 'это', 'эту', 'алло', 'буду', 'будь', 'бывь', 'была', 'были', 'было', 'быть', 'вами', 'ваша', 'ваше', 'ваши', 'ведь', 'весь', 'вниз', 'всем', 'всех', 'всею', 'года', 'году', 'даже', 'двух', 'день', 'если', 'есть', 'зато', 'кого', 'кому', 'куда', 'лишь', 'люди', 'мало', 'меля', 'меня', 'мимо', 'мира', 'мной', 'мною', 'мочь', 'надо', 'нами', 'наша', 'наше', 'наши', 'него', 'нему', 'ниже', 'ними', 'один', 'пока', 'пора', 'пять', 'рано', 'сама', 'сами', 'само', 'саму', 'свое', 'свои', 'свою', 'себе', 'себя', 'семь', 'стал', 'суть', 'твой', 'твоя', 'твоё', 'тебе', 'тебя', 'теми', 'того', 'тоже', 'тому', 'туда', 'хоть', 'хотя', 'чаще', 'чего', 'чему', 'чтоб', 'чуть', 'этим', 'этих', 'этой', 'этом', 'этот', 'более', 'будем', 'будет', 'будто', 'будут', 'вверх', 'вдали', 'вдруг', 'везде', 'внизу', 'время', 'всего', 'всеми', 'всему', 'всюду', 'давно', 'даром', 'долго', 'друго', 'жизнь', 'занят', 'затем', 'зачем', 'здесь', 'иметь', 'какая', 'какой', 'когда', 'кроме', 'лучше', 'между', 'менее', 'много', 'могут', 'может', 'можно', 'можхо', 'назад', 'низко', 'нужно', 'одной', 'около', 'опять', 'очень', 'перед', 'позже', 'после', 'потом', 'почти', 'пятый', 'разве', 'рядом', 'самим', 'самих', 'самой', 'самом', 'своей', 'своих', 'сеаой', 'снова', 'собой', 'собою', 'такая', 'также', 'такие', 'такое', 'такой', 'тобой', 'тобою', 'тогда', 'тысяч', 'уметь', 'часто', 'через', 'чтобы', 'шесть', 'этими', 'этого', 'этому', 'близко', 'больше', 'будете', 'будешь', 'бывает', 'важная', 'важное', 'важные', 'важный', 'вокруг', 'восемь', 'всегда', 'второй', 'далеко', 'дальше', 'девять', 'десять', 'должно', 'другая', 'другие', 'других', 'другое', 'другой', 'занята', 'занято', 'заняты', 'значит', 'именно', 'иногда', 'каждая', 'каждое', 'каждые', 'каждый', 'кругом', 'меньше', 'начала', 'нельзя', 'нибудь', 'никуда', 'ничего', 'обычно', 'однако', 'одного', 'отсюда', 'первый', 'потому', 'почему', 'просто', 'против', 'раньше', 'самими', 'самого', 'самому', 'своего', 'сейчас', 'сказал', 'совсем', 'теперь', 'только', 'третий', 'хорошо', 'хотеть', 'хочешь', 'четыре', 'шестой', 'восьмой', 'впрочем', 'времени', 'говорил', 'говорит', 'девятый', 'десятый', 'кажется', 'конечно', 'которая', 'которой', 'которые', 'который', 'которых', 'наверху', 'наконец', 'недавно', 'немного', 'нередко', 'никогда', 'однажды', 'посреди', 'сегодня', 'седьмой', 'сказала', 'сказать', 'сколько', 'слишком', 'сначала', 'спасибо', 'человек', 'двадцать', 'довольно', 'которого', 'наиболее', 'недалеко', 'особенно', 'отовсюду', 'двадцатый', 'миллионов', 'несколько', 'прекрасно', 'процентов', 'четвертый', 'двенадцать', 'непрерывно', 'пожалуйста', 'пятнадцать', 'семнадцать', 'тринадцать', 'двенадцатый', 'одиннадцать', 'пятнадцатый', 'семнадцатый', 'тринадцатый', 'шестнадцать', 'восемнадцать', 'девятнадцать', 'одиннадцатый', 'четырнадцать', 'шестнадцатый', 'восемнадцатый', 'девятнадцатый', 'действительно', 'четырнадцатый', 'многочисленная', 'многочисленное', 'многочисленные', 'многочисленный', "я", "мы", "ты", "oни", "нас", "вы", "тебя", "меня", "тебе", "вами", "мне", "тысяч", "котор"]

In [3]:
trigram_measures = nltk.collocations.TrigramAssocMeasures()
bigram_measures = nltk.collocations.BigramAssocMeasures()

In [14]:
def get_ngrams(n, path, filter_word):
    lemmatizer = WordNetLemmatizer()
    lt = [x[0] for x in os.walk(path)]

    dict_pronoun_words = {}
    bigrams_dict = {}

    for ftr_dir in lt[1:]:
        # Gets pronoun from file name
        pronoun = ftr_dir[17:]

        # Uses the json_dir variable to navigate to the dataset directory and 'get' all json files
        ftr_pattern = os.path.join(ftr_dir, '*.ftr')

        file_list = glob.glob(ftr_pattern)
        
        for i in file_list:
            bigram_measures = nltk.collocations.BigramAssocMeasures()
            flat_list = []
            read_ftr = pd.read_feather(i, columns=None, use_threads=True)
            ls = read_ftr["text"].tolist()

            flat_list = flat_list + [item for sublist in ls for item in sublist]
            dict_pronoun_words[pronoun] = flat_list
        
        if n == 2:
            # Bigrams
            for i in dict_pronoun_words:
                tokens = dict_pronoun_words.get(i)
                tokens = ' '.join([lemmatizer.lemmatize(w) for w in tokens])
                tokens= tokens.split()
                tokens = [wrd for wrd in tokens if wrd.lower() not in stop_words]
                finder = BigramCollocationFinder.from_words(tokens)
                # only bigrams that appear 3+ times
                finder.apply_freq_filter(1)
                # Ngrams with 'creature' as a member
                ngram_word_filter = lambda *w: filter_word not in w

                # only bigrams that contain the filter word
                finder.apply_ngram_filter(ngram_word_filter)
                #text_bigrams = bigrams(tokens)
                bigrams_dict[i] = finder.nbest(bigram_measures.likelihood_ratio, 10)
                print(bigrams_dict.get(i))
        
        if n == 3:
            for i in dict_pronoun_words:
                tokens = dict_pronoun_words.get(i)
                tokens = ' '.join([lemmatizer.lemmatize(w) for w in tokens])
                tokens= tokens.split()
                tokens = [wrd for wrd in tokens if wrd.lower() not in stop_words]
                finder = TrigramCollocationFinder.from_words(tokens)
                # only bigrams that appear 3+ times
                finder.apply_freq_filter(1)
                # Ngrams with 'creature' as a member
                ngram_word_filter = lambda *w: filter_word not in w

                # only bigrams that contain the filter word
                finder.apply_ngram_filter(ngram_word_filter)
                #text_bigrams = bigrams(tokens)
                bigrams_dict[i] = finder.nbest(trigram_measures.likelihood_ratio, 10)

    if n <= 3:
        return bigrams_dict

In [13]:
def get_ngrams_2(n, path, filter_word):
    lemmatizer = WordNetLemmatizer()

    dict_pronoun_words = {}
    bigrams_dict = {}

    # Uses the json_dir variable to navigate to the dataset directory and 'get' all json files
    ftr_dir = '../ftrs/individual-ftrs/'
    ftr_pattern = os.path.join(ftr_dir, '*.ftr')
    file_list = glob.glob(ftr_pattern)

    flat_list = []
    for i in file_list:
        bigram_measures = nltk.collocations.BigramAssocMeasures()
        read_ftr = pd.read_feather(i, columns=None, use_threads=True)
        ls = read_ftr["text"].tolist()

        flat_list = flat_list + [item for sublist in ls for item in sublist]
    
    if n == 2:
        # Bigrams
        for i in dict_pronoun_words:
            tokens = dict_pronoun_words.get(i)
            tokens = ' '.join([lemmatizer.lemmatize(w) for w in tokens])
            tokens= tokens.split()
            tokens = [wrd for wrd in tokens if wrd.lower() not in stop_words]
            finder = BigramCollocationFinder.from_words(tokens)
            # only bigrams that appear 3+ times
            finder.apply_freq_filter(1)
            # Ngrams with 'creature' as a member
            ngram_word_filter = lambda *w: filter_word not in w

            # only bigrams that contain the filter word
            finder.apply_ngram_filter(ngram_word_filter)
            #text_bigrams = bigrams(tokens)
            bigrams_dict[i] = finder.nbest(bigram_measures.likelihood_ratio, 10)
    
    if n == 3:
        flat_list = [wrd for wrd in flat_list if wrd.lower() not in stop_words]
        finder = TrigramCollocationFinder.from_words(flat_list)
        # only bigrams that appear 3+ times
        finder.apply_freq_filter(5)
        # Ngrams with 'creature' as a member
        ngram_word_filter = lambda *w: filter_word not in w

        # only bigrams that contain the filter word
        finder.apply_ngram_filter(ngram_word_filter)
        #text_bigrams = bigrams(tokens)
        trigrams = finder.nbest(trigram_measures.likelihood_ratio, 10)

    if n <= 3:
        return trigrams

In [15]:
trigrams_dic_знает = get_ngrams(3, '../ftrs/Pronouns/', 'знает')

In [20]:
for i in trigrams_dic_знает:
    print(i, "\n", trigrams_dic_знает.get(i))

oни 
 [('galinaotradnoe', 'знает', 'ге'), ('mpltnv', 'galinaotradnoe', 'знает'), ('знает', 'ге', 'долж')]
вас 
 [('mpltnv', 'galinaotradnoe', 'знает'), ('движк', 'ху', 'знает'), ('морожен', 'реж', 'знает'), ('знает', 'разыгрыва', 'чудесн'), ('galinaotradnoe', 'знает', 'ге'), ('знает', 'вейб', 'мног'), ('знает', 'масс', 'kovalyash'), ('общ', 'знает', 'вейб'), ('руководств', 'лгу', 'знает'), ('откуд', 'поч', 'знает')]
вы 
 [('mpltnv', 'galinaotradnoe', 'знает'), ('морожен', 'реж', 'знает'), ('движк', 'ху', 'знает'), ('знает', 'бренд', 'перекрашива'), ('знает', 'разыгрыва', 'чудесн'), ('galinaotradnoe', 'знает', 'ге'), ('знает', 'вейб', 'мног'), ('знает', 'масс', 'wlodzislavs'), ('реж', 'знает', 'масс'), ('откуд', 'поч', 'знает')]
их 
 [('движк', 'ху', 'знает'), ('знает', 'разыгрыва', 'чудесн'), ('знает', 'вейб', 'мног'), ('общ', 'знает', 'вейб'), ('помн', 'лгу', 'знает'), ('знает', 'реж', 'са'), ('ху', 'знает', 'реж'), ('дела', 'филарет', 'знает'), ('филарет', 'знает', 'дела'), ('откуд',

In [12]:
# for i in trigrams_dic_знает:
#     print(i, "\n", trigrams_dic_знает.get(i))
print(trigrams_dic_знает)

[('эфир', 'шлок', 'знает'), ('знает', 'домдв', 'домдв'), ('знает', 'нефтян', 'кризис'), ('раков', 'клеток', 'знает'), ('знает', 'via', 'youtube'), ('via', 'youtube', 'знает'), ('знает', 'друг', 'друг'), ('друг', 'друг', 'знает'), ('ал', 'савкин', 'знает'), ('семьдес', 'сем', 'знает')]


In [85]:
trigrams_dic_тоже = get_ngrams(3, '../ftrs/Pronouns/', 'тоже')

In [86]:
for i in trigrams_dic_тоже:
    print(i, "\n", trigrams_dic_тоже.get(i))

oни 
 []
вас 
 []
вы 
 []
их 
 []
меня 
 []
мне 
 []
мы 
 []
нас 
 []
тебя 
 []
ты 
 []
я 
 []


In [21]:
no_rt_bigrams_dic = get_ngrams(2, '../ftrs/Pronouns/')

In [ ]:
for i in bigrams_dic:
    print(i, "\n", no_rt_bigrams_dic.get(i))

 [('зомбиящик', 'крым'), ('коммент', 'естествен'), ('крым', 'фашист'), ('ленин', 'почтамт'), ('прощен', 'коммент'), ('фашист', 'цт'), ('плачут', 'прос'), ('почтамт', 'телефон'), ('прос', 'прощен'), ('телефон', 'телегаф')]

In [27]:
rt_bigrams_dic = get_ngrams(2, '../ftrs/ind-retweets/')

In [28]:
for i in rt_bigrams_dic:
    print(i, "\n", rt_bigrams_dic.get(i))

ets/I 
 [('вестпойнт', 'лозунг'), ('лгат', 'краст'), ('лозунг', 'лгат'), ('помпе', 'уч'), ('уч', 'вестпойнт'), ('alexeypushkov', 'помпе'), ('краст', 'мир'), ('rt', 'alexeypushkov'), ('мир', 'эт'), ('оставля', 'отзыв')]
